In [4]:
# boilerplate
import pandas as pd
import xlsxwriter

path_to_data = '../../clinical_trial_registries_data/'

In [5]:
# load data
df1 = pd.read_excel(path_to_data + 'ct_list_MC.xlsx')
df2 = pd.read_excel(path_to_data + 'ct_list_RS.xlsx')

In [6]:
# find trials to include
df1_includes = df1[df1['Screening'] == 'Include']
df2_includes = df2[df2['Screening'] == 'Include']
df_all_includes = pd.concat([df1_includes, df2_includes], axis=0)
include_filter = df_all_includes.duplicated(subset='TrialID', keep='first')
df_include = df_all_includes[include_filter]
df_include = df_include.sort_index()
df_include.drop(columns='Unnamed: 0', level=None, inplace=True)

In [7]:
# find trials to reconcile
df_reconcile = df_all_includes.drop_duplicates(subset='TrialID', keep=False)
df_reconcile = df_reconcile.sort_index()

df_reconcile.drop(columns=['Unnamed: 0', 'Screening'], level=None, inplace=True)
df_reconcile["Screening"] = " "


In [8]:
# save data
df_include.to_excel(path_to_data + 'ct_list_include.xlsx', index=True)
df_reconcile.to_excel(path_to_data + 'ct_list_reconcile.xlsx', index=True)

In [6]:
# Export to xlsx file
df_reconcile.to_excel(path_to_data + 'ct_list_reconcile.xlsx', engine='xlsxwriter') 

#Open it with xlsxwriter
writer = pd.ExcelWriter(path_to_data + 'ct_list_reconcile.xlsx', engine='xlsxwriter') 
df_reconcile.to_excel(writer, sheet_name='Sheet1')

#Assign the workbook and worksheet
workbook  = writer.book
worksheet = writer.sheets['Sheet1']

# add new column for screening
cell_format_colname = workbook.add_format({'bold': True})
worksheet.write(0, 5, 'Screening', cell_format_colname)


# reformat URL column so that the horizontal alignment is set to fill (stop text overflow)
cell_format_coldata = workbook.add_format()
cell_format_coldata.set_align('fill')

last_row = len(df_reconcile)

for row in range(0, last_row):

    trialid_ = df_reconcile.iloc[row, 0]
    title_ = df_reconcile.iloc[row, 1]
    register_ = df_reconcile.iloc[row, 2]
    url_ = df_reconcile.iloc[row, 3]

    row_reindex = row + 1

    worksheet.write(row_reindex, 1, trialid_, cell_format_coldata)
    worksheet.write(row_reindex, 2, title_, cell_format_coldata)
    worksheet.write(row_reindex, 3, register_, cell_format_coldata)
    worksheet.write(row_reindex, 4, url_, cell_format_coldata)

    worksheet.data_validation(row_reindex, 5, row_reindex, 5, {'validate': 'list',
                                  'source': ['Include',' Exclude']})


# close and save
workbook.close()